## 環境構築

In [15]:
!python --version

Python 3.10.0


日本語Wikipediaのデータをダウンロード

In [17]:
!curl -O https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3776M  100 3776M    0     0  3952k      0  0:16:18  0:16:18 --:--:-- 4018k    0  0:17:02 4021k  0  3949k      0  0:16:19  0:01:42  0:14:37 4037k     0  3885k      0  0:16:35  0:01:56  0:14:39 3293k  0  3846k      0  0:16:45  0:02:10  0:14:35 3985kM    0     0  3903k      0  0:16:30  0:03:14  0:13:16 4277k 0     0  3916k      0  0:16:27  0:03:24  0:13:03 4088k  3925k      0  0:16:25  0:03:45  0:12:40 3879k0     0  3925k      0  0:16:25  0:03:46  0:12:39 3889k915M    0     0  3933k      0  0:16:23  0:03:58  0:12:25 4022k   0  3939k      0  0:16:21  0:04:56  0:11:25 3672k    0  3967k      0  0:16:14  0:05:47  0:10:27 4035k   0  3974k      0  0:16:13  0:06:20  0:09:53 4229k   0     0  3970k      0  0:16:14  0:07:30  0:08:44 3995k   0     0  3972k      0  0:16:13  0:08:30  0:07:43 3806kM    0     0  3974k      0  0:16:13  0:08:39  

### wiki圧縮ファイルの解答

In [26]:
%pip install wikiextractor

Note: you may need to restart the kernel to use updated packages.


In [29]:
!python -m wikiextractor.WikiExtractor jawiki-latest-pages-articles.xml.bz2

INFO: Preprocessing 'jawiki-latest-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Preprocessed 1600000 pages
INFO: Preprocessed 1700000 pages
INFO: Preprocessed 1800000 pages
INFO: Preprocessed 1900000 pages
INFO: Preprocessed 2000000 pages
INFO: Preprocessed 2100000 pages
INFO: Preprocessed 2200000 pages
INFO: Preprocessed 2300000 pages
INFO: Preprocessed 2400000 pages
INFO: Preprocessed 2500000 pages
INFO: Preprocessed 2600000 pages
INFO: Preprocessed 2700000 pages
I

wikiをまとめる

In [41]:
!cat text/*/* > wiki.txt

不要なタグを削除

タグを除去

In [46]:
!sed -e 's/<[^>]*>//g' ./wiki.txt > ./wiki_notag.txt

sedコマンドに-iオプションをつけるとjupyterでうまく変換できなかったため、ターミナルから下記コマンドを実行

```bash
#全角カッコを半角に
$ sed -i -e 's/（/(/g' ./wiki_notag.txt && sed -i -e 's/）/)/g' ./wiki_notag.txt
 
#カッコ内を削除
$ sed -i -e 's/([^)]*)//g' ./wiki_notag.txt

#空白をすべて埋めたあと空行を削除
$ sed -i -e 's/ //g' ./wiki_notag.txt && sed -i -e '/^$/d' ./wiki_notag.txt
```

## MeCabで分かち書き

MeCabは既にインストール済みです。  
インストールは下記を参考  
https://www.mlab.im.dendai.ac.jp/~yamada/ir/MorphologicalAnalyzer/MeCab.html

macはbrewでインストールできる
```bash
brew install mecab
brew install mecab-ipadic
```


下記実行して分かち書き
```bash
mecab -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -Owakati wiki_notag.txt -o wiki_wakati.txt -b 163840
```

## Genismインストール

In [32]:
!pip install -U gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 618.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 4.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 32.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 30.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.0 MB/s eta 0:00:00


## ベクトルモデルの作成

In [51]:
from gensim.models import word2vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus('./wiki_wakati.txt')

model = word2vec.Word2Vec(sentences, vector_size=200, min_count=20, window=15)
model.wv.save_word2vec_format("./wiki.model", binary=True)

2023-10-21 00:45:36,105 : INFO : collecting all words and their counts
2023-10-21 00:45:36,114 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-10-21 00:45:56,909 : INFO : PROGRESS: at sentence #10000, processed 100000000 words, keeping 901408 word types
2023-10-21 00:46:18,513 : INFO : PROGRESS: at sentence #20000, processed 200000000 words, keeping 1433724 word types
2023-10-21 00:46:41,446 : INFO : PROGRESS: at sentence #30000, processed 300000000 words, keeping 1943959 word types
2023-10-21 00:47:04,258 : INFO : PROGRESS: at sentence #40000, processed 400000000 words, keeping 2453264 word types
2023-10-21 00:47:26,983 : INFO : PROGRESS: at sentence #50000, processed 500000000 words, keeping 2974064 word types
2023-10-21 00:47:48,920 : INFO : PROGRESS: at sentence #60000, processed 600000000 words, keeping 3489148 word types
2023-10-21 00:47:53,281 : INFO : collected 3580641 word types from a corpus of 619776064 raw words and 61978 sentences
2023-10-21

## モデルから類似の単語を検索

In [69]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format('wiki.model', binary=True)
results = wv.most_similar(positive=['旅行'])
for result in results:
  print(result)

2023-10-21 01:34:27,285 : INFO : loading projection weights from wiki.model
2023-10-21 01:34:30,374 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (413342, 200) matrix of type float32 from wiki.model', 'binary': True, 'encoding': 'utf8', 'datetime': '2023-10-21T01:34:30.374268', 'gensim': '4.3.2', 'python': '3.10.0 (default, Jul  8 2022, 09:56:12) [Clang 13.0.0 (clang-1300.0.27.3)]', 'platform': 'macOS-13.5.2-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


('新婚旅行', 0.6820679306983948)
('家族旅行', 0.6815314292907715)
('海外旅行', 0.6787171959877014)
('旅', 0.6646993160247803)
('滞在', 0.6621007323265076)
('国内旅行', 0.6549482941627502)
('一人旅', 0.6399331092834473)
('自転車旅行', 0.630429208278656)
('バックパッカー', 0.6293407678604126)
('旅先', 0.6195895075798035)
